In [ ]:
import cv2
import face_recognition
import pandas as pd

# Load known faces
known_faces = []
known_names = []
for name in ["person1", "person2"]:  # Add names of people here
    image = face_recognition.load_image_file(f"{name}.jpg")
    encoding = face_recognition.face_encodings(image)[0]
    known_faces.append(encoding)
    known_names.append(name)

# Initialize video capture
video_capture = cv2.VideoCapture(0)

while True:
    ret, frame = video_capture.read()
    rgb_frame = frame[:, :, ::-1]

    # Find faces in the video
    face_locations = face_recognition.face_locations(rgb_frame)
    face_encodings = face_recognition.face_encodings(rgb_frame, face_locations)

    for (top, right, bottom, left), face_encoding in zip(face_locations, face_encodings):
        matches = face_recognition.compare_faces(known_faces, face_encoding)

        if True in matches:
            first_match_index = matches.index(True)
            name = known_names[first_match_index]

            # Record attendance
            df = pd.read_csv('attendance.csv')
            if name not in df['Name'].values:
                df = df.append({'Name': name}, ignore_index=True)
                df.to_csv('attendance.csv', index=False)

    # Display the resulting image
    cv2.imshow('Video', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

video_capture.release()
cv2.destroyAllWindows()